In [1]:
import rain_and_wind_auto_park.database_stuff as dbs
from tqdm import tqdm
import pandas as pd
from datetime import datetime, timedelta
import time

In [2]:
engines = dbs.make_engines('../db_credentials.yml')
engine = engines['sandbox']

In [3]:
TABLE = dbs.make_copy_for_testing(
    engine, 
    copy_name='rain_and_wind_test_schedule',
    original_name='factdata.Schedule'
)

Let's look at the last 13 entries of our copy of the real schedule table.

We see e.g. a Startup at 2019-05-29 20:47:00  and a Shutdown at 2019-05-30 05:04:00.

In [4]:
dbs.show_last_n_rows(TABLE, engine, n=13)

,fScheduleID,fUser,fSourceName,fMeasurementTypeName
fStart,,,,
2019-05-29 02:55:49,32867,ToO,1ES 1959+650,Data
2019-05-29 03:25:00,32859,dorner,NVSS J220941-045111,Data
2019-05-29 05:04:00,32860,dorner,None,Shutdown
2019-05-29 20:47:00,32839,auto,None,Startup
2019-05-29 21:02:00,32840,auto,Mrk 421,Data
2019-05-29 22:43:00,32841,auto,Mrk 501,Data
2019-05-30 03:03:00,32842,auto,1ES 1959+650,Data
2019-05-30 05:04:00,32843,auto,None,Shutdown
2019-05-30 20:47:00,32861,auto,None,Startup


Let's assume we are currently observing Mrk 421 (i.e. it is after 21:02h) and it becomes stormy outside. Let's say it is 21:15h now.

In [5]:
now = datetime.fromisoformat('2019-05-29 21:15:00')

So now we need to insert a Suspend task at that specified time

In [6]:
dbs.insert_suspend_task_into_table_now(now, TABLE, engine)

Let's see if it worked, we need to see:
 * a Suspend task at the specified time 
 * AND a Resume task after the next Shutdown 

In [7]:
dbs.show_last_n_rows(TABLE, engine, n=13)

,fScheduleID,fUser,fSourceName,fMeasurementTypeName
fStart,,,,
2019-05-29 05:04:00,32860,dorner,None,Shutdown
2019-05-29 20:47:00,32839,auto,None,Startup
2019-05-29 21:02:00,32840,auto,Mrk 421,Data
2019-05-29 21:15:00,32868,rain_and_wind,None,Suspend
2019-05-29 22:43:00,32869,rain_and_wind,Mrk 501,Data
2019-05-30 03:03:00,32870,rain_and_wind,1ES 1959+650,Data
2019-05-30 05:04:00,32871,rain_and_wind,None,Shutdown
2019-05-30 05:05:00,32872,rain_and_wind,None,Resume
2019-05-30 20:47:00,32861,auto,None,Startup


Ok this worked, now lets assume it is 30min later and we detect that the storm has most likely stopped. So we want to insert a resume right now.

In [8]:
now = datetime.fromisoformat('2019-05-29 21:45:00')
dbs.insert_resume_task_into_table_now(now, TABLE, engine)

And now let us check if it worked. We would expect the Resume, which was formerly after the next Shutdown to be moved forward, so it is right now in the schedule.

In [9]:
dbs.show_last_n_rows(TABLE, engine, n=13)

,fScheduleID,fUser,fSourceName,fMeasurementTypeName
fStart,,,,
2019-05-29 05:04:00,32860,dorner,None,Shutdown
2019-05-29 20:47:00,32839,auto,None,Startup
2019-05-29 21:02:00,32840,auto,Mrk 421,Data
2019-05-29 21:15:00,32868,rain_and_wind,None,Suspend
2019-05-29 21:45:00,32873,rain_and_wind,None,Resume
2019-05-29 22:43:00,32874,rain_and_wind,Mrk 501,Data
2019-05-30 03:03:00,32875,rain_and_wind,1ES 1959+650,Data
2019-05-30 05:04:00,32876,rain_and_wind,None,Shutdown
2019-05-30 20:47:00,32861,auto,None,Startup


Ok, so this worked so far. In the rest of this document, I want to adress as few more exotic cases

## Duplicate Suspend at the same time?

What happens, if for some strange reason our program wants to insert suspend two times in a row .. basically at the same time (rounded to a second, this might be possible...).
Do we crash? Do we get multiple inserts? What happens?

First lets reset our testing copy

In [10]:
TABLE = dbs.make_copy_for_testing(
    engine, 
    copy_name='rain_and_wind_test_schedule',
    original_name='factdata.Schedule'
)
now = datetime.fromisoformat('2019-05-30 21:15:00')
dbs.insert_suspend_task_into_table_now(now, TABLE, engine)
dbs.show_last_n_rows(TABLE, engine, n=8)

,fScheduleID,fUser,fSourceName,fMeasurementTypeName
fStart,,,,
2019-05-30 05:04:00,32843,auto,None,Shutdown
2019-05-30 20:47:00,32861,auto,None,Startup
2019-05-30 21:02:00,32862,auto,Mrk 421,Data
2019-05-30 21:15:00,32868,rain_and_wind,None,Suspend
2019-05-30 22:39:00,32869,rain_and_wind,Mrk 501,Data
2019-05-31 02:59:00,32870,rain_and_wind,1ES 1959+650,Data
2019-05-31 05:04:00,32871,rain_and_wind,None,Shutdown
2019-05-31 05:05:00,32872,rain_and_wind,None,Resume


Ok, this is the normal insert and now we inset again at the same time...

In [11]:
dbs.insert_suspend_task_into_table_now(now, TABLE, engine)
dbs.show_last_n_rows(TABLE, engine, n=8)

,fScheduleID,fUser,fSourceName,fMeasurementTypeName
fStart,,,,
2019-05-30 05:04:00,32843,auto,None,Shutdown
2019-05-30 20:47:00,32861,auto,None,Startup
2019-05-30 21:02:00,32862,auto,Mrk 421,Data
2019-05-30 21:15:00,32873,rain_and_wind,None,Suspend
2019-05-30 22:39:00,32874,rain_and_wind,Mrk 501,Data
2019-05-31 02:59:00,32875,rain_and_wind,1ES 1959+650,Data
2019-05-31 05:04:00,32876,rain_and_wind,None,Shutdown
2019-05-31 05:05:00,32877,rain_and_wind,None,Resume


Ok. no crash, that is good I guess. And also no duplicate entry. Good?

Next we look at the same for Resume, one second later

In [12]:
now = datetime.fromisoformat('2019-05-30 21:15:01')
dbs.insert_resume_task_into_table_now(now, TABLE, engine)
dbs.show_last_n_rows(TABLE, engine, n=8)

,fScheduleID,fUser,fSourceName,fMeasurementTypeName
fStart,,,,
2019-05-30 05:04:00,32843,auto,None,Shutdown
2019-05-30 20:47:00,32861,auto,None,Startup
2019-05-30 21:02:00,32862,auto,Mrk 421,Data
2019-05-30 21:15:00,32873,rain_and_wind,None,Suspend
2019-05-30 21:15:01,32878,rain_and_wind,None,Resume
2019-05-30 22:39:00,32879,rain_and_wind,Mrk 501,Data
2019-05-31 02:59:00,32880,rain_and_wind,1ES 1959+650,Data
2019-05-31 05:04:00,32881,rain_and_wind,None,Shutdown


Okay now the "duplicate" call in the exact same time

In [13]:
dbs.insert_resume_task_into_table_now(now, TABLE, engine)
dbs.show_last_n_rows(TABLE, engine, n=8)

,fScheduleID,fUser,fSourceName,fMeasurementTypeName
fStart,,,,
2019-05-30 05:04:00,32843,auto,None,Shutdown
2019-05-30 20:47:00,32861,auto,None,Startup
2019-05-30 21:02:00,32862,auto,Mrk 421,Data
2019-05-30 21:15:00,32873,rain_and_wind,None,Suspend
2019-05-30 21:15:01,32882,rain_and_wind,None,Resume
2019-05-30 22:39:00,32883,rain_and_wind,Mrk 501,Data
2019-05-31 02:59:00,32884,rain_and_wind,1ES 1959+650,Data
2019-05-31 05:04:00,32885,rain_and_wind,None,Shutdown


All good, right? No crash, no duplicate entry, all fine.

Now the next case.

## Suspend and Resume at the same time?

In [14]:
TABLE = dbs.make_copy_for_testing(
    engine, 
    copy_name='rain_and_wind_test_schedule',
    original_name='factdata.Schedule'
)
now = datetime.fromisoformat('2019-05-30 21:15:00')
dbs.insert_suspend_task_into_table_now(now, TABLE, engine)
dbs.show_last_n_rows(TABLE, engine, n=8)

,fScheduleID,fUser,fSourceName,fMeasurementTypeName
fStart,,,,
2019-05-30 05:04:00,32843,auto,None,Shutdown
2019-05-30 20:47:00,32861,auto,None,Startup
2019-05-30 21:02:00,32862,auto,Mrk 421,Data
2019-05-30 21:15:00,32868,rain_and_wind,None,Suspend
2019-05-30 22:39:00,32869,rain_and_wind,Mrk 501,Data
2019-05-31 02:59:00,32870,rain_and_wind,1ES 1959+650,Data
2019-05-31 05:04:00,32871,rain_and_wind,None,Shutdown
2019-05-31 05:05:00,32872,rain_and_wind,None,Resume


In [15]:
dbs.insert_resume_task_into_table_now(now, TABLE, engine)
dbs.show_last_n_rows(TABLE, engine, n=8)

,fScheduleID,fUser,fSourceName,fMeasurementTypeName
fStart,,,,
2019-05-30 03:03:00,32842,auto,1ES 1959+650,Data
2019-05-30 05:04:00,32843,auto,None,Shutdown
2019-05-30 20:47:00,32861,auto,None,Startup
2019-05-30 21:02:00,32862,auto,Mrk 421,Data
2019-05-30 21:15:00,32873,rain_and_wind,None,Resume
2019-05-30 22:39:00,32874,rain_and_wind,Mrk 501,Data
2019-05-31 02:59:00,32875,rain_and_wind,1ES 1959+650,Data
2019-05-31 05:04:00,32876,rain_and_wind,None,Shutdown


Okay, so the Resume basically overwrites the Suspend. good enough for us?

Next case:

## Multiple Suspends?

In [16]:
TABLE = dbs.make_copy_for_testing(
    engine, 
    copy_name='rain_and_wind_test_schedule',
    original_name='factdata.Schedule'
)
now = datetime.fromisoformat('2019-05-30 21:15:00')
dbs.insert_suspend_task_into_table_now(now, TABLE, engine)
now = datetime.fromisoformat('2019-05-30 21:15:01')
dbs.insert_suspend_task_into_table_now(now, TABLE, engine)
now = datetime.fromisoformat('2019-05-30 21:15:02')
dbs.insert_suspend_task_into_table_now(now, TABLE, engine)

dbs.show_last_n_rows(TABLE, engine, n=8)

,fScheduleID,fUser,fSourceName,fMeasurementTypeName
fStart,,,,
2019-05-30 21:02:00,32862,auto,Mrk 421,Data
2019-05-30 21:15:00,32868,rain_and_wind,None,Suspend
2019-05-30 21:15:01,32873,rain_and_wind,None,Suspend
2019-05-30 21:15:02,32878,rain_and_wind,None,Suspend
2019-05-30 22:39:00,32879,rain_and_wind,Mrk 501,Data
2019-05-31 02:59:00,32880,rain_and_wind,1ES 1959+650,Data
2019-05-31 05:04:00,32881,rain_and_wind,None,Shutdown
2019-05-31 05:05:00,32882,rain_and_wind,None,Resume


Not nice ... but as expected, when we tell it insert 3 suspends ... it does so.

What about ...

## Multiple Resumes?

In [17]:
TABLE = dbs.make_copy_for_testing(
    engine, 
    copy_name='rain_and_wind_test_schedule',
    original_name='factdata.Schedule'
)
now = datetime.fromisoformat('2019-05-30 21:15:00')
dbs.insert_resume_task_into_table_now(now, TABLE, engine)
now = datetime.fromisoformat('2019-05-30 21:15:01')
dbs.insert_resume_task_into_table_now(now, TABLE, engine)
now = datetime.fromisoformat('2019-05-30 21:15:02')
dbs.insert_resume_task_into_table_now(now, TABLE, engine)

dbs.show_last_n_rows(TABLE, engine, n=8)

,fScheduleID,fUser,fSourceName,fMeasurementTypeName
fStart,,,,
2019-05-30 20:47:00,32861,auto,None,Startup
2019-05-30 21:02:00,32862,auto,Mrk 421,Data
2019-05-30 21:15:00,32868,rain_and_wind,None,Resume
2019-05-30 21:15:01,32872,rain_and_wind,None,Resume
2019-05-30 21:15:02,32876,rain_and_wind,None,Resume
2019-05-30 22:39:00,32877,rain_and_wind,Mrk 501,Data
2019-05-31 02:59:00,32878,rain_and_wind,1ES 1959+650,Data
2019-05-31 05:04:00,32879,rain_and_wind,None,Shutdown


Also good, it is not nice to be able to create multiple Resumes, but it does exactly as we tell it.

Last case, what if we insert a Suspend or Resume task, exactly at the time of an existing task.
Let's take the Mrk 501 for example:

## Insert at exact time of existing task

In [18]:
TABLE = dbs.make_copy_for_testing(
    engine, 
    copy_name='rain_and_wind_test_schedule',
    original_name='factdata.Schedule'
)
now = datetime.fromisoformat('2019-05-30 22:39:00')
dbs.insert_suspend_task_into_table_now(now, TABLE, engine)

dbs.show_last_n_rows(TABLE, engine, n=8)

,fScheduleID,fUser,fSourceName,fMeasurementTypeName
fStart,,,,
2019-05-30 05:04:00,32843,auto,None,Shutdown
2019-05-30 20:47:00,32861,auto,None,Startup
2019-05-30 21:02:00,32862,auto,Mrk 421,Data
2019-05-30 22:39:00,32869,rain_and_wind,Mrk 501,Data
2019-05-30 22:39:01,32868,rain_and_wind,None,Suspend
2019-05-31 02:59:00,32870,rain_and_wind,1ES 1959+650,Data
2019-05-31 05:04:00,32871,rain_and_wind,None,Shutdown
2019-05-31 05:05:00,32872,rain_and_wind,None,Resume


Okay, so turns out the `fStart` field is defined to be a unique index of the table, so 

In [19]:
now = datetime.fromisoformat('2019-05-31 02:59:00')
dbs.insert_resume_task_into_table_now(now, TABLE, engine)
dbs.show_last_n_rows(TABLE, engine, n=8)

,fScheduleID,fUser,fSourceName,fMeasurementTypeName
fStart,,,,
2019-05-30 05:04:00,32843,auto,None,Shutdown
2019-05-30 20:47:00,32861,auto,None,Startup
2019-05-30 21:02:00,32862,auto,Mrk 421,Data
2019-05-30 22:39:00,32869,rain_and_wind,Mrk 501,Data
2019-05-30 22:39:01,32868,rain_and_wind,None,Suspend
2019-05-31 02:59:00,32874,rain_and_wind,1ES 1959+650,Data
2019-05-31 02:59:01,32873,rain_and_wind,None,Resume
2019-05-31 05:04:00,32875,rain_and_wind,None,Shutdown


Okay like this?